# Spark 101

In this lesson we will cover the basics of working with spark dataframes, and show how spark dataframes are different from the pandas dataframes we have been working with.

While spark dataframes might superficially look like pandas dataframes, and even share some of the same methods and syntax, it is important to keep in mind they are 2 seperate types of objects, and, while spark and pandas code might look superficially similar, it tends to be semantically very different.

We'll begin by creating the spark session:

In [ ]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

## Creating Dataframes

Spark can convert any pandas dataframe into a spark dataframe with a simple method call. For this lesson, we will use this functionality to demonstrate the differences between spark and pandas dataframes and explore how to work with spark dataframes.

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(456)

pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), group=np.random.choice(list("abc"), 20))
)
pandas_dataframe

Here we start with a simple pandas dataset, and now we will convert it to a spark dataframe:

In [ ]:
df = spark.createDataFrame(pandas_dataframe)
df

Notice that, while we do see the column names, we don't see the data in the dataframe like we would with a pandas dataframe. This is because spark is lazy, in that it won't show us values until it has to. For the purposes of looking at the first few rows of our data, we can use the `.show` method.

In [ ]:
df.show(5)

Like pandas dataframes, spark dataframes have a `.describe` method:

In [ ]:
df.describe()

Which, also like pandas, returns another dataframe. However, since this is a spark dataframe, we have to explicitly show it.

In [ ]:
df.describe().show()

By default spark will show the first 20 rows, but we can specify how many we want by passing a number to `.show`.

Observing the shape of our spark dataframe does require a little bit of creativity:

In [ ]:
print("DataFrame shape: ", df.count(), " x ", len(df.columns))

Let's use some different data so that we have a more robust dataset:

In [ ]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

Let's look at another difference from pandas:

In [ ]:
mpg.hwy

While this expression would produce a Series of values from a pandas dataframe, for a spark dataframe this produces a Column object, which is an object that represents a vertical slice of a dataframe, but does not contain the data itself.

One way to use our column objects is to use them in combination with the `.select` method. `.select` is very powerful, and lets us specify what data we want to see in the resulting dataframe.

In [ ]:
mpg.select(mpg.hwy, mpg.cty, mpg.model)

Again, notice that we don't see any data, instead we see the new dataframe that is produced. To see the actual data, we'll again need to use `.show`

In [ ]:
mpg.select(mpg.hwy, mpg.cty, mpg.model).show(10)

Our column objects support a number of operations, including the arithmetic operators:

In [ ]:
mpg.hwy + 1

Here we get back a column that represents the values from the original hwy column with 1 added to them. To actually see this data, we'd need to select it and show the dataframe.

In [ ]:
mpg.select(mpg.hwy, mpg.hwy + 1).show(5)

Once we have a column object, we can use the `.alias` method to rename it:

In [ ]:
mpg.select(mpg.hwy.alias("highway_mileage")).show(5)

We can also store column objects in variables and reference them.

In [ ]:
col1 = mpg.hwy.alias("highway_mileage")
col2 = (mpg.hwy / 2).alias("highway_mileage_halved")
mpg.select(col1, col2).show(5)

## Other ways to create columns

In addition to the syntax we've seen above, we can create columns with the `col` and `expr` functions from `pyspark.sql.functions` module.

In [ ]:
from pyspark.sql.functions import col, expr

In [ ]:
col("hwy")

The output above is somewhat meaningless. Without attaching this col to a source, it's just a lazy object:

In [ ]:
test_col = col("hwy")

In [ ]:
test_col

We can mix and match the syntax we use, and the column object produced by the `col` function is the same as the the previous column object we saw.

Here `col("hwy")` finds a match in the mpg data when we use `.select()`

In [ ]:
avg_column = (col("hwy") + col("cty")) / 2

mpg.select(
    col("hwy").alias("highway_mileage"),
    mpg.cty.alias("city_mileage"),
    avg_column.alias("avg_mileage"),
).show(5)

We created a variable named `avg_column` that represents the average of the highway and city mileage of each vehicle. This variable is created by using the `col` function to produce pyspark Column objects and using the arithmetic operators to combine them.

Next we select the original highway and city mileage columns, in addition to our new average mileage column. We demonstrate the `col` function to select the `hwy` column and refer to the city mileage column with the `mpg.cty` syntax we saw previously. We also give all of our columns more readable aliases before showing the resulting dataframe.

Notice something curious when we display `avg_column` by itself:

In [ ]:
avg_column

It's like a recipe or formula for spark to use when it has something to match `hwy` and `cty` to. Again, Spark is **lazy** and tries to limit the amount of information stored in memory. It accomplishes this by storing the instructions and only deploying those instructions when absolutely necessary.

## `expr`
The `expr` function is more powerful than `col`. It does everything `col` does and more. `expr` returns the same type of column object, but allows us to express manipulations to the column within the string that defines the column.

In [ ]:
mpg.select(
    expr("hwy"),  # the same as `col`
    expr("hwy + 1"),  # an arithmetic expression
    expr("hwy AS highway_mileage"),  # using an alias
    expr("hwy + 1 AS highway_incremented"),  # a combination of the above
).show(5)

There's a lot of redundancy in Spark...

In [ ]:
mpg.select(
    mpg.hwy.alias("highway"),
    col("hwy").alias("highway"),
    expr("hwy").alias("highway"),
    expr("hwy AS highway"),
).show(5)

Note that all the columns created above are identical, and which syntax to use is merely a style choice.

## Type Casting

We can view the types of the column in our dataframe in one of two ways:

In [ ]:
mpg.dtypes

In [ ]:
mpg.printSchema()

Both provide the same information.

To convert from one type to another, we can use the `.cast` method on a column.

In [ ]:
mpg.select(mpg.hwy.cast("string")).printSchema()

Note that if a value is not able to be converted, it will be replaced with null:

In [ ]:
mpg.select(mpg.model, mpg.model.cast("int")).show(5)

# ------------------------------------------------------------------------
## Exercise 1:

Within your `codeup-data-science` directory, create a new repo named `spark-exercises`. This will be where you do your work for this module. Create a repository on GitHub with the same name, and link your local repository to GitHub.

Save this work in your `spark-exercises` repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named `spark101` for this exercise.

Create a spark data frame that contains your favorite programming languages.

- Create a dataframe with one column named `language`
> Hint: Start with a pandas dataframe. Maybe use a dictionary?
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

# ------------------------------------------------------------------------

## Spark SQL

As we've seen through the column definitions, spark is very flexible and allows us many different ways to express ourselves. Another way that is fairly different than what we've seen above is through spark SQL, which lets us write SQL queries against our spark dataframes.

In order to start using spark SQL, we'll first "register" the table with spark:




In [ ]:
mpg.createOrReplaceTempView("mpg")

Now we can write a sql query against the `mpg` table:

In [ ]:
spark.sql(
    """
SELECT hwy, cty, (hwy + cty) / 2 AS avg
FROM mpg
"""
)

Notice that the resulting value is another dataframe. As we know, in order to view the values in a dataframe, we need to use `.show`

In [ ]:
spark.sql(
    """
SELECT hwy, cty, (hwy + cty) / 2 AS avg
FROM mpg
"""
).show(5)

It is worth noting that all of the methods for creating / manipulating dataframes outlined above are the same in terms of performance as well. All of the resulting dataframes get turned into the same spark code that gets executed on the JVM, so it really is just a style choice as to which to use.

## Basic Built-in Functions

We've used the `col` and `expr` functions, but there are many other functions within the `pyspark.sql.functions` module, all of which operate on pyspark dataframe columns. Here we'll demonstrate several:

- `concat`: to concatenate strings
- `sum`: to sum a group
- `avg`: to take the average of a group
- `min`: to find the minimum
- `max`: to find the maximum

### WARNING:
**Note that importing the `sum` function directly will override python's built-in `sum` function.** 

This means you will get an error if you try to `sum` a list of numbers, because `sum` will refernce the **pyspark** `sum` function, which works with pyspark dataframe columns, while the built-in `sum` function works with lists of numbers. **The same holds true for the built in `min` and `max` functions.**

In [ ]:
# Note: The pyspark avg and mean functions are aliases of eachother
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean

In this lesson we will explicitly import any functions from pyspark.sql.functions that we use, but it very common to see something like:

> `from pyspark.sql.functions import *`

which will import *all* of the functions from the `pyspark.sql.functions` module.

In [ ]:
mpg.select(
    sum(mpg.hwy) / count(mpg.hwy).alias("average_1"),
    avg(mpg.hwy).alias("average_2"),
    min(mpg.hwy),
    max(mpg.hwy),
).show()

In [ ]:
mpg.columns

### `concat`

We can create custom values by concatenating columns together:

In [ ]:
mpg.select(concat(mpg.manufacturer, mpg.model)).show(5)

What if we want to concatenate a string that's not already in our data? In order to use a string literal as part of our select, we'll need to use the `lit` function, otherwise spark will try to resolve our string as a column.

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
mpg.select(concat(mpg.cyl, lit(" cylinders"))).show(5)

In the example above, we select the concatenation of the number of cylinders (the value from the `cyl` column) and the string literal " cylinders".

In [ ]:
# What happens if we don't use `lit()`?
mpg.select(concat(mpg.cyl, " cylinders")).show(5)

## More `pyspark` Functions for String Manipulation

Let's take a look at a couple more functions for string manipulation.

In [ ]:
from pyspark.sql.functions import regexp_extract, regexp_replace

In order to demonstrate these functions we'll create a dataframe with some text data.

In [ ]:
textdf = spark.createDataFrame(
    pd.DataFrame(
        {
            "address": [
                "600 Navarro St ste 600, San Antonio, TX 78205",
                "3130 Broadway St, San Antonio, TX 78209",
                "303 Pearl Pkwy, San Antonio, TX 78215",
                "1255 SW Loop 410, San Antonio, TX 78227",
            ]
        }
    )
)

textdf.show(truncate=False)

The `regexp_extract` function lets us specify a regular expression with at least one capture group, and create a new column based on the contents of a capture group.

In [ ]:
textdf.select(
    "address",
    regexp_extract("address", r"^(\d+)", 1).alias("street_no"),
    regexp_extract("address", r"^\d+\s([\w\s]+?),", 1).alias("street"),
).show(truncate=False)

In the example above, the first argument to `regexp_extract` is the name of the string column to extract from, the second argument is the regular expression itself, and the last argument specifies which capture group we want to use. If, for example, our regular expression had 2 capture groups in it and we wanted the contents of the 2nd group, we would specify a 2 here.

In addition to `regexp_extract`, `regexp_replace` lets us make substitutions based on a regular expression:

In [ ]:
textdf.select(
    "address",
    regexp_replace("address", r"^.*?,\s*", "").alias("city_state_zip"),
).show(truncate=False)

In our example above, we obtain just the city, state, and zip code of the address by replacing everything up to the first comma with an empty string.

## `.filter` and `.where`

Spark provides two dataframe methods, `.filter` and `.where`, which both allow us to select a subset of the rows of our dataframe.

In [ ]:
mpg.filter(mpg.cyl == 4).where(mpg["class"] == "subcompact").show()

In [ ]:
mpg.filter(mpg.cyl == 4).filter(mpg["class"] == "subcompact").show()

In [ ]:
mpg.where(mpg.cyl == 4).where(mpg["class"] == "subcompact").show()

They give the same result. There is no difference. "filter" is the standard Scala name for such a function and "where" is more familiar to people who use SQL.

## `when()` and `.otherwise()`

Similar to:
- an `IF` in Excel,
- `CASE...WHEN` in SQL, 
- `np.where` in python, 

Spark provides a `when()` function.

The `when` function lets us specify a condition, and **a value to produce** if that condition is true:

In [ ]:
from pyspark.sql.functions import when

In [ ]:
mpg.select(mpg.hwy, when(mpg.hwy > 25, "good_mileage").alias("mpg_desc")).show(
    12
)

Notice here that if the condition we specified is false, `null` will be produced. Instead of null, we can specify a value to use if our condition is false with the `.otherwise` method.

In [ ]:
mpg.select(
    mpg.hwy,
    when(mpg.hwy > 25, "good_mileage")
    .otherwise("bad_mileage")
    .alias("mpg_desc"),
).show(12)

To specify multiple conditions, we can chain `.when` calls. The first condition that is met will be the value that is used, and if none of the conditions are met the value specified in the `.otherwise` will be used (or `null` if you don't provide a `.otherwise`).

In [ ]:
mpg.select(
    mpg.displ,
    (
        when(mpg.displ < 2, "small")
        .when(mpg.displ < 3, "medium")
        .otherwise("large")
        .alias("engine_size")
    ),
).show(10)

Notice here that a car with a `displ` of 1.8 matches both conditions we specified, but `small` is produced because it is associated with the first matching condition. For any value between 2 and 3, `medium` will be produced, and anything larger than 3 will produce `large`

# ------------------------------------------------------------------------
## Exercise 2:

Load the `mpg` dataset as a spark dataframe.

a. Create 1 column of output that contains a message like the one below for each record:

    The 1999 audi a4 has a 4 cylinder engine.

> Hint: You will need to concatenate values that already exist in the data with string literals

b. Transform the trans column so that it only contains either manual or auto.

> Hint: Consider spark string methods and `when().otherwise()` chaining
# ------------------------------------------------------------------------

## `withColumn()`

We can add a new column to our dataframe using the withColumn() function. The first argument is the name of the new column, and the second argument is the forumla or value that we want in our column.

In [ ]:
mpg.withColumn("is_car", lit("yes")).show()

We can also use `.withColumn()` to perform operations using multiple columns as input:

In [ ]:
mpg.select(mpg.manufacturer, mpg.model,mpg.hwy, mpg.cty).withColumn("ratio", col('hwy') / col('cty')).show()

**NOTE:** If I use `.select()`, I must include the columns used to create the `.withColumn()` result

In [ ]:
mpg.select(mpg.manufacturer, mpg.model).withColumn("ratio", col('hwy') / col('cty')).show()

## Sorting and Ordering

Spark lets us sort the rows in our dataframe by one or multiple columns with two methods: `.sort`, and `.orderBy`. `.sort` and `.orderBy` are aliases of each other and do the exact same thing. Like other methods we've seen, `.sort` takes in a Column object or a string that is the name of a column.

In [ ]:
mpg.sort(mpg.hwy).show(8)

By default, values are sorted in ascending order. To sort in descending order, we can use the `.desc` method on any Column object, or the `desc` function from `pyspark.sql.functions`.

In [ ]:
from pyspark.sql.functions import asc, desc

In [ ]:
# Using the .desc() method
mpg.sort(mpg.hwy.desc()).show(5)

In [ ]:
# Using the col() function with .desc() method
mpg.sort(col("hwy").desc()).show(5)

In [ ]:
# Using the desc() function
mpg.sort(desc("hwy")).show(5)

To specify sorting by multiple columns, we provide each column as a separate argument to `.sort`.

Here we will first reverse alphabetically by the vehicle's class, then by the number of cylinders from lowest to highest, then by the vehicle's highway mileage, from greatest to smallest:

In [ ]:
mpg.sort(desc("class"), mpg.cyl.asc(), col("hwy").desc()).show()

## Grouping and Aggregating

To aggregate our data by group, we can use the `.groupBy` method. Like with `.select`, we can pass either Column objects or strings that are column names to `.groupBy`. All of the expressions below are equivalent.

In [ ]:
mpg.groupBy(mpg.cyl)
mpg.groupBy(col("cyl"))
mpg.groupBy("cyl")

Once the data is grouped, we need to specify an aggregation. We can use one of the aggregate functions we imported earlier (`sum`, `avg`, `min`, `max`, etc.) along with a column:

In [ ]:
mpg.groupBy(mpg.cyl).agg(avg(mpg.cty), avg(mpg.hwy)).show()

To group by multiple columns, pass each of the columns a a separate argument to `.groupBy` (Note that this is different from pandas, where we would need to pass a list).

In [ ]:
mpg.groupBy("cyl", "class").agg(avg(mpg.cty), avg(mpg.hwy)).show()

In addition to `.groupBy`, we can use `.rollup`, which will do the same aggregations, but will also include the overall total:

In [ ]:
mpg.rollup("cyl").count().sort("cyl").show()

Here the null value in `cyl` indicates the total count.

In [ ]:
mpg.rollup("cyl").agg(expr("avg(hwy)")).sort("cyl").show()

And in the example above, the null row represents the overall average highway mileage.

In [ ]:
mpg.rollup("cyl", "class").mean("hwy").sort(col("cyl"), col("class")).show()

## Crosstabs and Pivot Tables

In addition to groupby, spark provides a couple other ways to do aggregation. One of which is `.crosstab`. This is very similary to pandas `.crosstab` function, in that it calculates the number of occurances of each unique value from the two passed columns:

In [ ]:
mpg.crosstab("class", "cyl").show()

`.crosstab` simply does counts, if we want a different aggregation, we can use `.pivot`. For example, to find the average highway mileage for each combination of car class and number of cylinders, we could write the following:

In [ ]:
mpg.groupby("class").pivot("cyl").mean("hwy").show()

Here the unique values from the column we group by will be the rows in the resulting dataframe, and the unique values from the column we pivot on will become the columns. The values in each cell will be equal to the aggregation we specified over the group of values defined by the intersection of the rows and the columns.

# ------------------------------------------------------------------------
## Exercise 3: 

Load the `tips` dataset as a spark dataframe.

a. What percentage of observations are smokers?
> Hint: `.groupBy()` and `.withColumn()` are useful functions here

b. Create a column that contains the tip percentage
> Hint: `.withColumn()` is useful here

c. Calculate the average tip percentage for each combination of sex and smoker.
> Hint: Chain additional functions off the answer to part b 

# ------------------------------------------------------------------------

## Handling Missing Data

Let's take a look at how spark handles missing data. First we'll create a dataframe that has a few missing values:

In [ ]:
df = spark.createDataFrame(
    pd.DataFrame(
        {"x": [1, 2, np.nan, 4, 5, np.nan], "y": [np.nan, 0, 0, 3, 1, np.nan]}
    )
)
df.show()

Spark provides two main ways to deal with missing values:

- `.fill`: to replace missing values with a specified value
- `.drop`: to drop rows containing missing values
Both methods are accessed through the `.na` property. We'll look at some examples below:

In [ ]:
df.na.drop().show()

In [ ]:
df.na.fill(0).show()

For both methods, we can specify that we only want to fill or drop values in a specific column with a second argument:

In [ ]:
df.na.fill(0, subset="x").show()

Notice that above the na values in the x column were filled with 0, but the na values in y were left alone.

In [ ]:
df.na.drop(subset="y").show()

In the example above, the rows that had an na value for the y column were dropped, but the rows with na values for only the x column are still present.

## Explaining DataFrame Transformations

The `.explain` method will show us how spark is thinking about our dataframe.

For our basic example, we see that there is only a single step:

In [ ]:
mpg.explain()

When we add a select function, there is another step after "Scan ExistingRDD", a "Project" that contains the names of the columns we are looking for:

In [ ]:
mpg.select(mpg.cyl, mpg.hwy).explain()

Here we are doing a more advanced select calculation, but this is still just a single step to spark.

In [ ]:
mpg.select(((mpg.cyl + mpg.hwy) / 2).alias("avg_mpg")).explain()m

A filter is also a single step:

In [ ]:
mpg.filter(mpg.cyl == 6).explain()

Without reading ahead, do you think the execution plan for the two dataframes below will be the same or not?

In [ ]:
mpg.select("cyl", "hwy").filter(expr("cyl = 6")).explain()
mpg.filter(expr("cyl = 6")).select("cyl", "hwy").explain()

Notice that even though we specified the transformations (`.select` and `.filter`) in a different order, we end up with the same output when we call `.explain`. This is because spark will look at our dataframe and transform it into the most efficient representation possible.

In [ ]:
mpg.selectExpr("cyl + 3 * 16 / 4 + 19 AS unused", "hwy").select(
    "hwy"
).explain()

Notice above that we had 2 separate select statements, but spark condensed this down to a single Project, as it is smart enough to realize that it doesn't actually need to do all the arithmetic we specified in the first select, since we arent using that value later on.

Now our execution plan gets more complicated:

In [ ]:
mpg.select(min(mpg.cyl)).explain()

This is because in steps prior, we were applying transformations that applied to each row individually. To calculate a minimum, we have to look at all the rows in the dataset to find the smallest.

In [ ]:
mpg.groupby(mpg.cyl).agg(min(mpg.hwy), max(mpg.hwy)).explain()


In [ ]:
(
    mpg.select(col("cyl"), expr("(cty + hwy) / 2 AS avg_mpg"))
    .filter(expr('class == "compact"'))
    .groupby("cyl")
    .agg(min("avg_mpg"), avg("avg_mpg"), max("avg_mpg"))
    .explain()
)

## More Dataframe Manipulation Examples

Let's take a look at some more examples of working with spark dataframes. For these examples, we'll be working with a dataset of observations of the weather in seattle.

In [ ]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

Let's print out the number of rows and columns in our dataset:

In [ ]:
print(weather.count(), "rows", len(weather.columns), "columns")

Let's first find the dates where the data starts and stops:

In [ ]:
min_date, max_date = weather.select(min("date"), max("date")).first()
min_date, max_date

We used `.select` to select the minimum date and the maximum date. `.first` returns us the first row of our results, which consists of two value, and so can be unpacked into the min_date and max_date variables.

Next we will combine the temp max and min columns into a single column, `temp_avg`:

In [ ]:
weather = weather.withColumn(
    "temp_avg", expr("ROUND(temp_min + temp_max) / 2")
).drop("temp_max", "temp_min")
weather.show(6)

Now we will calculate the total amount of rainfall for each month. We'll do this by first creating a month column, then grouping by the month, and finally, aggregating by taking the sum of the precipitation. To do this we will need to use the `month` function.

In [ ]:
from pyspark.sql.functions import month, year, quarter

In [ ]:
(
    weather.withColumn("month", month("date"))
    .groupBy("month")
    .agg(sum("precipitation").alias("total_rainfall"))
    .sort("month")
    .show()
)

The `.sort` at the end isn't necessary, but presents that data in a friendlier way.

Let's now take a look at the average temperature for each type of weather in December 2013:

In [ ]:
(
    weather.filter(month("date") == 12)
    .filter(year("date") == 2013)
    .groupBy("weather")
    .agg(mean("temp_avg"))
    .show()
)

Here we first have a couple of `.filter` calls in order to restrict our data to December of 2013. We then group by the weather column, and lastly, aggregate by taking the average of our `temp_avg` column. The combination of group by and agg will calculate the average temperature for each unique value of the `weather` column.

Let's now find out how many days had freezing temperatures in each month of 2013.

In [ ]:
(
    weather.filter(year("date") == 2013)
    .withColumn("freezing_temps", (weather.temp_avg <= 0).cast("int"))
    .withColumn("month", month("date"))
    .groupBy("month")
    .agg(sum("freezing_temps").alias("no_of_days_with_freezing_temps"))
    .sort("month")
    .show()
)

One last example, let's calculate the average temperature for each quarter of each year:

In [ ]:
(
    weather.withColumn("quarter", quarter("date"))
    .withColumn("year", year("date"))
    .groupBy("year", "quarter")
    .agg(mean("temp_avg").alias("temp_avg"))
    .sort("year", "quarter")
    .show()
)

Here we created the `quarter` and `year` columns, then grouped by these two new columns, and took the average temperature as our aggregate. Lastly, we sorted by the year and quarter for presentation purposes.

We could also use a pivot table like this:

In [ ]:
(
    weather.withColumn("quarter", quarter("date"))
    .withColumn("year", year("date"))
    .groupBy("quarter")
    .pivot("year")
    .agg(expr("ROUND(MEAN(temp_avg), 2) AS temp_avg"))
    .sort("quarter")
    .show()
)

Here instead of grouping by two columns, we grouped by the first column and pivoted on the other column.

## Joins

Like pandas and sql, spark has functionality that lets us combine two tabular datasets, known as a join.

We'll start by creating some data that we can join together:

In [ ]:
users = spark.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4, 5, 6],
            "name": ["bob", "joe", "sally", "adam", "jane", "mike"],
            "role_id": [1, 2, 3, 3, np.nan, np.nan],
        }
    )
)
roles = spark.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4],
            "name": ["admin", "author", "reviewer", "commenter"],
        }
    )
)
print("--- users ---")
users.show()
print("--- roles ---")
roles.show()

To join two dataframes together, we'll need to call the `.join` method on one of them and supply the other as an argument. In addition, we'll need to supply the condition on which we are joining. In our case, we are joining where the `role_id` column on the users table is equal to the `id` column on the roles table.

In [ ]:
users.join(roles, on=users.role_id == roles.id).show()

By default, spark will perform an inner join, meaning that records from both dataframes will have a match with the other. We can also specify either a left or a right join, which will keep all of the records from either the left or right side, even if those records don't have a match with the other dataframe.

In [ ]:
users.join(roles, on=users.role_id == roles.id, how="left").show()

In [ ]:
users.join(roles, on=users.role_id == roles.id, how="right").show()

Notice that examples above have a duplicate `id` column. There are several ways we could go about dealing with this:

- alias each dataframe + explicitly select columns after joining (this could also be implemented with spark SQL)
- rename duplicated columns before merging
- drop duplicated columns after the merge (`.drop(right.id)`)

## Visualization (or Lack Therof)

Spark does not provide a way to do visualization with their dataframes. To visualize data from spark, you should use the `.toPandas` method on a spark dataframe to convert it to a pandas dataframe, then visualize as you normally would.

> Converting a spark dataframe to a pandas dataframe will pull all the data into memory, so make sure you have enough available memory to do so.

# ------------------------------------------------------------------------
## Exercise 4:

Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to fahrenheit.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).